<a href="https://colab.research.google.com/github/enoosoft/ai_fsec/blob/master/colab/4_mnist_nn_deep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
# Lab 10 MNIST and softmax
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import random

In [13]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
random.seed(777)
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [14]:
# parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [15]:
# MNIST dataset
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

In [16]:
# dataset loader
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [17]:
# nn layers
linear1 = torch.nn.Linear(784, 512, bias=True) # 깊고 넓게 노드 볼륨 확장
linear2 = torch.nn.Linear(512, 512, bias=True)
linear3 = torch.nn.Linear(512, 512, bias=True)
linear5 = torch.nn.Linear(512, 10, bias=True)
relu = torch.nn.ReLU()

In [18]:
# xavier initialization
torch.nn.init.xavier_uniform_(linear1.weight)
torch.nn.init.xavier_uniform_(linear2.weight)
torch.nn.init.xavier_uniform_(linear3.weight)
torch.nn.init.xavier_uniform_(linear5.weight)

Parameter containing:
tensor([[-0.0207, -0.0504,  0.0425,  ...,  0.0668,  0.0628,  0.0494],
        [ 0.0006, -0.1034, -0.0434,  ..., -0.1033, -0.0928, -0.0020],
        [-0.0566,  0.1013, -0.0628,  ..., -0.0625,  0.0688, -0.0043],
        ...,
        [-0.0294, -0.0180,  0.0634,  ...,  0.0530,  0.0959, -0.0384],
        [ 0.0819,  0.0163, -0.0410,  ...,  0.0132, -0.0256,  0.0817],
        [-0.0236, -0.0281,  0.0883,  ..., -0.0216,  0.0474, -0.0854]],
       requires_grad=True)

In [19]:
# model
model = torch.nn.Sequential(linear1, relu, linear2, relu, linear3, relu,  linear5).to(device)

In [20]:
# define cost/loss & optimizer
criterion = torch.nn.CrossEntropyLoss().to(device)    # Softmax is internally computed.
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
total_batch = len(data_loader)
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader:
        # reshape input image into [batch_size by 784]
        # label is not one-hot encoded
        X = X.view(-1, 28 * 28).to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning finished') # 과적합: 무조건 깊고 넓게 한다고 학습이 잘되는 것은 아니다.

Epoch: 0001 cost = 0.202576309
Epoch: 0002 cost = 0.081490196
Epoch: 0003 cost = 0.057076886
Epoch: 0004 cost = 0.042506780
Epoch: 0005 cost = 0.035414502
Epoch: 0006 cost = 0.027644560
Epoch: 0007 cost = 0.024341373
Epoch: 0008 cost = 0.023156025
Epoch: 0009 cost = 0.019536555
Epoch: 0010 cost = 0.020321460
Epoch: 0011 cost = 0.016065363
Epoch: 0012 cost = 0.016054258
Epoch: 0013 cost = 0.014333991


In [ ]:
# Test the model using test sets
with torch.no_grad():
    X_test = mnist_test.test_data.view(-1, 28 * 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

    # Get one and predict
    r = random.randint(0, len(mnist_test) - 1)
    X_single_data = mnist_test.test_data[r:r + 1].view(-1, 28 * 28).float().to(device)
    Y_single_data = mnist_test.test_labels[r:r + 1].to(device)

    print('Label: ', Y_single_data.item())
    single_prediction = model(X_single_data)
    print('Prediction: ', torch.argmax(single_prediction, 1).item())